# Neural Networks

For: Pao Pao

The idea for this file is to train a a ANN and an GRU model given the dataset. The data files you will need to import is unfortunately not ready. But for now, write and test the code using `model_building_data.csv` which is provided in the data folder. Keep in mind that the final training/testing files will have more fields.

Neural Network based models cant handle missing data. Also, the range should be between -1 to 1 where possible. Also, an ANN is *not* a time series model. So lagged variables are needed.

Use Keras for both models. The task itself is too complex so Keras is more suitable.

Last thing to keep in mind is, some rows might have missing revenue but non-missing CAR etc. If you will drop NaNs, drop for each y values differently to prevent unnecessary data loss.

Tune all parameters using Optuna if possible (or use 3-fold cv) with the timesplit function like in assignment 1.

This file should save the output of the prediction in the format:

| ticker | quarter_year  | log_revenue_prediction | CAR_prediction |
|--------|---------------|------------------------|----------------|
| BAC    | Q1 2001       | 123                    | 0.5            |
| JPM    | Q1 2001       | 456                    | 0.8            |
| WFC    | Q1 2001       | 789                    | 0.25           |